In [17]:
pip install -r requirements.txt

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/95/fc/661a7f06e8b7d48fcbd3f55423b7ff1ac3ce59526f146fda87a1e1788ee4/datasets-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/ad/97/b8253236dfedb9094f4273393a3fd03997da81f27f15822e56128da894ae/gensim-4.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/7b/0e/25d6b5678ed3c7e12bc94d047d0e9492e89cc78b7ea0034ac0f1cf2ff304/scikit_learn-1.4.1.p


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
import numpy as np
import re
from datasets import load_dataset
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity
import ast

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vishesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
dataset = load_dataset("cfilt/iitb-english-hindi")

data_list = []
for item in dataset['train']:
  data_list.append({'english': item['translation']['en'], 'hindi': item['translation']['hi']})

df = pd.DataFrame(data_list)
df

,english,hindi
0,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4,A list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
...,...,...
1659078,"The Prime Minister, Shri Narendra Modi has con...",प्रधानमंत्री श्री नरेन्द्र मोदी ने बिहार के मु...
1659079,"In a tweet, the Prime Minister said, congratul...","एक ट्वीट में प्रधानमंत्री ने कहा, बिहार के मुख..."
1659080,I also congratulate all those who took oath as...,"मैं उन सभी को भी बधाई देता हूं, जिन्होंने बिहा..."
1659081,The NDA family will work together for the prog...,एनडीए परिवार बिहार की प्रगति के लिए साथ मिलकर ...


In [14]:
df2 = df[:829541]
df2

,english,hindi
0,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4,A list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
...,...,...
829536,consequently,परिणामस्वरूप
829537,accordingly,परिणामस्वरूप
829538,consequently,परिणामतः
829539,accordingly,परिणामतः


In [15]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[\r\n\t]', ' ', text)

    text = text.lower()

    tokens = word_tokenize(text)

    return ' '.join(tokens)

df2['english'] = df2['english'].apply(preprocess_text)
df2['hindi'] = df2['hindi'].apply(preprocess_text)
df2

C:\Users\Vishesh\AppData\Local\Temp\ipykernel_10472\3632964918.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['english'] = df2['english'].apply(preprocess_text)
C:\Users\Vishesh\AppData\Local\Temp\ipykernel_10472\3632964918.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['hindi'] = df2['hindi'].apply(preprocess_text)


,english,hindi
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4,a list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
...,...,...
829536,consequently,परिणामस्वरूप
829537,accordingly,परिणामस्वरूप
829538,consequently,परिणामतः
829539,accordingly,परिणामतः


In [16]:
df2['english_tokens'] = df2['english'].apply(word_tokenize)
df2['hindi_tokens'] = df2['hindi'].apply(word_tokenize) 

wmodel = Word2Vec(sentences=df2['english_tokens'].tolist() + df2['hindi_tokens'].tolist(), vector_size=100, window=5, min_count=1, workers=4)

def sentence_to_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(vectors, axis=0)

df2['english_vector'] = df2['english_tokens'].apply(lambda tokens: sentence_to_vector(tokens, wmodel))
df2['hindi_vector'] = df2['hindi_tokens'].apply(lambda tokens: sentence_to_vector(tokens, wmodel))
df2

C:\Users\Vishesh\AppData\Local\Temp\ipykernel_10472\2607873966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['english_tokens'] = df2['english'].apply(word_tokenize)
C:\Users\Vishesh\AppData\Local\Temp\ipykernel_10472\2607873966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['hindi_tokens'] = df2['hindi'].apply(word_tokenize)
C:\Users\Vishesh\AppData\Local\Temp\ipykernel_10472\2607873966.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,english,hindi,english_tokens,hindi_tokens,english_vector,hindi_vector
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,"[give, your, application, an, accessibility, w...","[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[1.0720485, 0.47643247, 0.46114084, -0.8396702...","[1.301204, 0.42534363, -0.68393457, -0.6074015..."
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,"[accerciser, accessibility, explorer]","[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[-0.26665387, -0.11733174, -0.099012874, 0.295...","[-0.15797089, -0.0780096, 0.1200985, 0.1923387..."
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,"[the, default, plugin, layout, for, the, botto...","[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[-0.12577052, -0.41703582, 0.7442633, -0.02574...","[0.36487222, 0.26381615, -0.7793632, -0.834450..."
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,"[the, default, plugin, layout, for, the, top, ...","[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[-0.037192043, -0.6551722, 0.7360175, -0.15139...","[0.3856779, 0.2556599, -0.7930813, -0.69814426..."
4,a list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,"[a, list, of, plugins, that, are, disabled, by...","[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[0.18132427, -0.9112907, 1.4789109, -0.8693911...","[1.3015217, 0.4625628, -1.1959735, -0.6886162,..."
...,...,...,...,...,...,...
829536,consequently,परिणामस्वरूप,[consequently],[परिणामस्वरूप],"[-0.34096956, 0.0032430303, 0.09304967, 0.2308...","[0.13698584, 0.49705565, -0.42426777, 1.174666..."
829537,accordingly,परिणामस्वरूप,[accordingly],[परिणामस्वरूप],"[-0.19228035, -0.10233332, -0.03283243, 0.2339...","[0.13698584, 0.49705565, -0.42426777, 1.174666..."
829538,consequently,परिणामतः,[consequently],[परिणामतः],"[-0.34096956, 0.0032430303, 0.09304967, 0.2308...","[-0.06768929, 0.083966985, 0.17150335, -0.0013..."
829539,accordingly,परिणामतः,[accordingly],[परिणामतः],"[-0.19228035, -0.10233332, -0.03283243, 0.2339...","[-0.06768929, 0.083966985, 0.17150335, -0.0013..."


In [23]:
df2.to_csv('data.csv', index=False)

In [18]:
df2.to_pickle('data.pkl')

## Run from here

In [2]:
import pandas as pd
import numpy as np
import re
from datasets import load_dataset
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity
import ast

c:\Users\Vishesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vishesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df3 = pd.read_pickle("data.pkl")
df3

,english,hindi,english_tokens,hindi_tokens,english_vector,hindi_vector
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,"[give, your, application, an, accessibility, w...","[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[1.0720485, 0.47643247, 0.46114084, -0.8396702...","[1.301204, 0.42534363, -0.68393457, -0.6074015..."
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,"[accerciser, accessibility, explorer]","[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[-0.26665387, -0.11733174, -0.099012874, 0.295...","[-0.15797089, -0.0780096, 0.1200985, 0.1923387..."
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,"[the, default, plugin, layout, for, the, botto...","[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[-0.12577052, -0.41703582, 0.7442633, -0.02574...","[0.36487222, 0.26381615, -0.7793632, -0.834450..."
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,"[the, default, plugin, layout, for, the, top, ...","[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[-0.037192043, -0.6551722, 0.7360175, -0.15139...","[0.3856779, 0.2556599, -0.7930813, -0.69814426..."
4,a list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,"[a, list, of, plugins, that, are, disabled, by...","[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[0.18132427, -0.9112907, 1.4789109, -0.8693911...","[1.3015217, 0.4625628, -1.1959735, -0.6886162,..."
...,...,...,...,...,...,...
829536,consequently,परिणामस्वरूप,[consequently],[परिणामस्वरूप],"[-0.34096956, 0.0032430303, 0.09304967, 0.2308...","[0.13698584, 0.49705565, -0.42426777, 1.174666..."
829537,accordingly,परिणामस्वरूप,[accordingly],[परिणामस्वरूप],"[-0.19228035, -0.10233332, -0.03283243, 0.2339...","[0.13698584, 0.49705565, -0.42426777, 1.174666..."
829538,consequently,परिणामतः,[consequently],[परिणामतः],"[-0.34096956, 0.0032430303, 0.09304967, 0.2308...","[-0.06768929, 0.083966985, 0.17150335, -0.0013..."
829539,accordingly,परिणामतः,[accordingly],[परिणामतः],"[-0.19228035, -0.10233332, -0.03283243, 0.2339...","[-0.06768929, 0.083966985, 0.17150335, -0.0013..."


In [4]:
embedding_size = 100

# df3['english_vector'] = df3['english_vector'].apply(lambda x: np.array(ast.literal_eval(x)))
# df3['hindi_vector'] = df3['hindi_vector'].apply(lambda x: np.array(ast.literal_eval(x)))

df3['english_vector'] = df3['english_vector'].apply(lambda x: np.array(x))
df3['hindi_vector'] = df3['hindi_vector'].apply(lambda x: np.array(x))

max_length_eng = max(df3['english_vector'].apply(len))
max_length_hin = max(df3['hindi_vector'].apply(len))

In [5]:
english_vectors_padded = pad_sequences(df3['english_vector'].tolist(), maxlen=max_length_eng, padding='post', dtype='float32')
english_vectors_padded = english_vectors_padded.reshape(-1, 829541, embedding_size)

hindi_vectors_padded = pad_sequences(df3['hindi_vector'].tolist(), maxlen=max_length_hin, padding='post', dtype='float32')
hindi_vectors_padded = hindi_vectors_padded.reshape(-1, 829541, embedding_size)

df3

,english,hindi,english_tokens,hindi_tokens,english_vector,hindi_vector
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,"[give, your, application, an, accessibility, w...","[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[1.0720485, 0.47643247, 0.46114084, -0.8396702...","[1.301204, 0.42534363, -0.68393457, -0.6074015..."
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,"[accerciser, accessibility, explorer]","[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[-0.26665387, -0.11733174, -0.099012874, 0.295...","[-0.15797089, -0.0780096, 0.1200985, 0.1923387..."
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,"[the, default, plugin, layout, for, the, botto...","[निचले, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[-0.12577052, -0.41703582, 0.7442633, -0.02574...","[0.36487222, 0.26381615, -0.7793632, -0.834450..."
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,"[the, default, plugin, layout, for, the, top, ...","[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लग-इन, खाका]","[-0.037192043, -0.6551722, 0.7360175, -0.15139...","[0.3856779, 0.2556599, -0.7930813, -0.69814426..."
4,a list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,"[a, list, of, plugins, that, are, disabled, by...","[उन, प्लग-इनों, की, सूची, जिन्हें, डिफोल्ट, रू...","[0.18132427, -0.9112907, 1.4789109, -0.8693911...","[1.3015217, 0.4625628, -1.1959735, -0.6886162,..."
...,...,...,...,...,...,...
829536,consequently,परिणामस्वरूप,[consequently],[परिणामस्वरूप],"[-0.34096956, 0.0032430303, 0.09304967, 0.2308...","[0.13698584, 0.49705565, -0.42426777, 1.174666..."
829537,accordingly,परिणामस्वरूप,[accordingly],[परिणामस्वरूप],"[-0.19228035, -0.10233332, -0.03283243, 0.2339...","[0.13698584, 0.49705565, -0.42426777, 1.174666..."
829538,consequently,परिणामतः,[consequently],[परिणामतः],"[-0.34096956, 0.0032430303, 0.09304967, 0.2308...","[-0.06768929, 0.083966985, 0.17150335, -0.0013..."
829539,accordingly,परिणामतः,[accordingly],[परिणामतः],"[-0.19228035, -0.10233332, -0.03283243, 0.2339...","[-0.06768929, 0.083966985, 0.17150335, -0.0013..."


In [6]:
lstm_units = 256

encoder_inputs = Input(shape=(None, embedding_size))
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, embedding_size))
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(embedding_size, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

In [7]:
lstm_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
lstm_model.compile(optimizer=Adam(), loss='mse')

lstm_model.fit([english_vectors_padded, hindi_vectors_padded[:, :-1]], hindi_vectors_padded[:, 1:], epochs=10, batch_size=32)

Epoch 1/10


In [ ]:
def find_nearest_word(vector, model):
    max_similarity = -1
    nearest_word = None
    for word in model.wv.index_to_key:
        similarity = cosine_similarity([vector], [model.wv[word]])[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            nearest_word = word
    return nearest_word

In [ ]:
def translate_english_to_hindi(english_sentence, model, lstm_model):
    tokens = word_tokenize(english_sentence)
    english_vectors = [sentence_to_vector([token], model) for token in tokens]
    english_vectors_padded = np.array([np.pad(english_vectors, ((0, max_length_eng - len(english_vectors)), (0, 0)), 'constant')])

    predicted_hindi_vectors = lstm_model.predict([english_vectors_padded, np.zeros((1, max_length_hin - 1, embedding_size))])[0]
    predicted_hindi_words = [find_nearest_word(vector, model) for vector in predicted_hindi_vectors]

    hindi_sentence = ' '.join(predicted_hindi_words).capitalize()

    return hindi_sentence

In [ ]:
english_sentence = "Hello, how are you?"
hindi_translation = translate_english_to_hindi(english_sentence, wmodel, lstm_model)
print("English:", english_sentence)
print("Hindi Translation:", hindi_translation)

1/1 [==============================] - 1s 1s/step
English: Hello, how are you?
Hindi Translation: खोलें खोलें अगला अगला सफेदसिर सफेदसिर सफेदसिर सफेदसिर सफेदसिर सफेदसिर सफेदसिर सफेदसिर सफेदसिर सफेदसिर aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot aisleriot 